### 🎯 Module Overview
This module covers everything you need to know about parsing and ingesting data for RAG systems, from basic text files to complex PDFs and databases. We'll use LangChain v0.3 and explore each technique with practical examples.

Table of Contents

- Introduction to Data Ingestion
- Text Files (.txt)
- PDF Documents
- Microsoft Word Documents
- CSV and Excel Files
- JSON and Structured Data
- Web Scraping
- Databases (SQL)
- Audio and Video Transcripts
- Advanced Techniques
- Best Practices

## Introduction to Data Ingestion

In [5]:
from typing import List,Dict,Any
import pandas as pd

In [6]:
from langchain_core.documents import Document
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)

print("SetUp Completed")

SetUp Completed


## Understanding Document Struture In LangChain

In [11]:
## create a simple document
doc = Document(
    page_content="This is the main text that will embedded and searched",
    metadata={
        "source":"example.txt",
        "page":1,
        "author":"The Abishek Khasre",
        "date_creatted":"2025-09-06",
        "custom_feild" : "any value"
    }
)

print("Document Structure")

print(f"Content : {doc.page_content}")
print(f"MetaData : {doc.metadata}")

Document Structure
Content : This is the main text that will embedded and searched
MetaData : {'source': 'example.txt', 'page': 1, 'author': 'The Abishek Khasre', 'date_creatted': '2025-09-06', 'custom_feild': 'any value'}


### TextLoader - Read Single File

In [15]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./data/text_files/python_intro.txt",encoding="utf-8")
docs = loader.load()

In [21]:
print(docs)
print(type(docs))
print(type(docs[0]))

[Document(metadata={'source': './data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]
<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [22]:
documents=loader.load()
print(f"📄 Loaded {len(documents)} document")
print(f"Content preview: {documents[0].page_content[:100]}...")
print(f"Metadata: {documents[0].metadata}")

📄 Loaded 1 document
Content preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...
Metadata: {'source': './data/text_files/python_intro.txt'}


## DirectoryLoader - Multiple Text Files

In [25]:
from langchain_community.document_loaders import DirectoryLoader

dir_loader = DirectoryLoader(
    "data/text_files",
    glob="**/*.txt" , ##pattern to match files
    loader_cls=TextLoader, ##loader class to use
    loader_kwargs={"encoding" : "utf-8"}
)


documents=dir_loader.load()

print(f"📁 Loaded {len(documents)} documents")
for i, doc in enumerate(documents):
    print(f"\nDocument {i+1}:")
    print(f"  Source: {doc.metadata['source']}")
    print(f"  Length: {len(doc.page_content)} characters")


# 📊 Analysis
print("\n📊 DirectoryLoader Characteristics:")
print("✅ Advantages:")
print("  - Loads multiple files at once")
print("  - Supports glob patterns")
print("  - Progress tracking")
print("  - Recursive directory scanning")

print("\n❌ Disadvantages:")
print("  - All files must be same type")
print("  - Limited error handling per file")
print("  - Can be memory intensive for large directories")

📁 Loaded 2 documents

Document 1:
  Source: data\text_files\machine_learning.txt
  Length: 575 characters

Document 2:
  Source: data\text_files\python_intro.txt
  Length: 489 characters

📊 DirectoryLoader Characteristics:
✅ Advantages:
  - Loads multiple files at once
  - Supports glob patterns
  - Progress tracking
  - Recursive directory scanning

❌ Disadvantages:
  - All files must be same type
  - Limited error handling per file
  - Can be memory intensive for large directories


### Text Splitting Strategies

In [28]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter
)

print(documents)

[Document(metadata={'source': 'data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '), Document(metadata={'source': 'data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprog

In [30]:
## Method 1-Charachter Text Splitter

text = documents[0].page_content
text


'Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '

In [35]:
print("1. Character Text Splitter")
char_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size = 200,
    chunk_overlap = 20,
    length_function = len
)

splitted_docs = char_splitter.split_documents(documents)

for i,doc in enumerate(splitted_docs):
    print(f"\nDocument {i+1}:")
    print(f"  Source: {doc.metadata['source']}")
    print(f"  Length: {len(doc.page_content)} characters")

1. Character Text Splitter

Document 1:
  Source: data\text_files\machine_learning.txt
  Length: 121 characters

Document 2:
  Source: data\text_files\machine_learning.txt
  Length: 179 characters

Document 3:
  Source: data\text_files\machine_learning.txt
  Length: 177 characters

Document 4:
  Source: data\text_files\machine_learning.txt
  Length: 85 characters

Document 5:
  Source: data\text_files\python_intro.txt
  Length: 130 characters

Document 6:
  Source: data\text_files\python_intro.txt
  Length: 200 characters

Document 7:
  Source: data\text_files\python_intro.txt
  Length: 154 characters


In [40]:
print("2. Recursive Character Text Splitter")
Recurchar_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n" , "\n" ," ",""],
    chunk_size = 200,
    chunk_overlap = 20,
    length_function = len
)

splitted_docs = Recurchar_splitter.split_documents(documents)

for i,doc in enumerate(splitted_docs):
    print(f"\nDocument {i+1}:")
    print(f"  Source: {doc.metadata['source']}")
    print(f"  Length: {len(doc.page_content)} characters")

2. Recursive Character Text Splitter

Document 1:
  Source: data\text_files\machine_learning.txt
  Length: 23 characters

Document 2:
  Source: data\text_files\machine_learning.txt
  Length: 193 characters

Document 3:
  Source: data\text_files\machine_learning.txt
  Length: 56 characters

Document 4:
  Source: data\text_files\machine_learning.txt
  Length: 138 characters

Document 5:
  Source: data\text_files\machine_learning.txt
  Length: 65 characters

Document 6:
  Source: data\text_files\machine_learning.txt
  Length: 85 characters

Document 7:
  Source: data\text_files\python_intro.txt
  Length: 31 characters

Document 8:
  Source: data\text_files\python_intro.txt
  Length: 196 characters

Document 9:
  Source: data\text_files\python_intro.txt
  Length: 35 characters

Document 10:
  Source: data\text_files\python_intro.txt
  Length: 124 characters

Document 11:
  Source: data\text_files\python_intro.txt
  Length: 96 characters


In [41]:
for doc in splitted_docs:
    print(doc.page_content)
    print("-"*10)

Machine Learning Basics
----------
Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
----------
that can access data and use it to learn for themselves.
----------
Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
----------
3. Reinforcement Learning: Learning through rewards and penalties
----------
Applications include image recognition, speech processing, and recommendation systems
----------
Python Programming Introduction
----------
Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
----------
programming languages in the world.
----------
Key Features:
- Easy to learn and use
- Extensive standard library
- C

In [42]:
# Create text without natural break points
simple_text = "This is sentence one and it is quite long. This is sentence two and it is also quite long. This is sentence three which is even longer than the others. This is sentence four. This is sentence five. This is sentence six."


In [43]:
# Method 3: Token-based splitting
print("\n3 TOKEN TEXT SPLITTER")
token_splitter = TokenTextSplitter(
    chunk_size=50,  # Size in tokens (not characters)
    chunk_overlap=10
)

token_chunks = token_splitter.split_text(text)
print(f"Created {len(token_chunks)} chunks")
print(f"First chunk: {token_chunks[0][:100]}...")


3️3 TOKEN TEXT SPLITTER
Created 3 chunks
First chunk: Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables system...


In [44]:
# 📊 Comparison
print("\n📊 Text Splitting Methods Comparison:")
print("\nCharacterTextSplitter:")
print("  ✅ Simple and predictable")
print("  ✅ Good for structured text")
print("  ❌ May break mid-sentence")
print("  Use when: Text has clear delimiters")

print("\nRecursiveCharacterTextSplitter:")
print("  ✅ Respects text structure")
print("  ✅ Tries multiple separators")
print("  ✅ Best general-purpose splitter")
print("  ❌ Slightly more complex")
print("  Use when: Default choice for most texts")

print("\nTokenTextSplitter:")
print("  ✅ Respects model token limits")
print("  ✅ More accurate for embeddings")
print("  ❌ Slower than character-based")
print("  Use when: Working with token-limited models")


📊 Text Splitting Methods Comparison:

CharacterTextSplitter:
  ✅ Simple and predictable
  ✅ Good for structured text
  ❌ May break mid-sentence
  Use when: Text has clear delimiters

RecursiveCharacterTextSplitter:
  ✅ Respects text structure
  ✅ Tries multiple separators
  ✅ Best general-purpose splitter
  ❌ Slightly more complex
  Use when: Default choice for most texts

TokenTextSplitter:
  ✅ Respects model token limits
  ✅ More accurate for embeddings
  ❌ Slower than character-based
  Use when: Working with token-limited models
